```bash
az group create \
  --name deploy-test-rg \
  --location eastus
```

Please note this requires installation of databricks auto-install extensions in the cli. Premium is chosen to enable Unity Catalog.
```bash
az databricks workspace create \
  --resource-group deploy-test-rg \
  --name deploy-test-workspace \
  --location eastus \
  --sku premium

# Get workspace URL
export WORKSPACE_URL=$(az databricks workspace show \
  -g deploy-test-rg \
  -n deploy-test-workspace \
  --query workspaceUrl -o tsv)

echo "Workspace URL: https://$WORKSPACE_URL"


```

```bash
# Set authentication variables. You will need to wait for deployment to complete to create your pat token.
export DATABRICKS_HOST="https://$WORKSPACE_URL"
export DATABRICKS_TOKEN="your-pat-token"  # Create in workspace UI: Settings > Developer > Access Tokens
```


```bash
curl -X POST "$DATABRICKS_HOST/api/2.1/unity-catalog/catalogs" \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"name": "rearcquest", "comment": "Rearc Quest data catalog"}'
```

```bash
curl -X POST "$DATABRICKS_HOST/api/2.1/unity-catalog/schemas" \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"name": "raw", "catalog_name": "rearcquest", "comment": "Raw data from sources"}'
```

```bash
curl -X POST "$DATABRICKS_HOST/api/2.1/unity-catalog/schemas" \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"name": "default", "catalog_name": "rearcquest", "comment": "Reports and analysis tables"}'
```

```bash
curl -X POST "$DATABRICKS_HOST/api/2.1/unity-catalog/volumes" \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"catalog_name": "rearcquest", "schema_name": "raw", "name": "bls_data", "volume_type": "MANAGED", "comment": "BLS productivity data files"}'
```

```bash
curl -X POST "$DATABRICKS_HOST/api/2.1/unity-catalog/volumes" \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"catalog_name": "rearcquest", "schema_name": "raw", "name": "population", "volume_type": "MANAGED", "comment": "Population data from API"}'
```

```bash
# Create Job: Daily Data Sync (Parts 1, 2, & 3)
curl -X POST "$DATABRICKS_HOST/api/2.1/jobs/create" \
  -H "Authorization: Bearer $DATABRICKS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{
    "name": "Rearc Quest - Daily Data Sync",
    "tasks": [
      {
        "task_key": "sync_bls_data",
        "notebook_task": {
          "notebook_path": "/Users/rebeccalwatkins@gmail.com/rearc_quest/Databricks Version/Rearc Databricks - Section 1",
          "base_parameters": {
            "catalog": "rearcquest",
            "schema": "raw",
            "volume": "bls_data",
            "sourceUrl": "https://download.bls.gov/pub/time.series/pr/"
          }
        },
        "new_cluster": {
          "spark_version": "15.4.x-scala2.12",
          "node_type_id": "Standard_DS3_v2",
          "num_workers": 2
        }
      },
      {
        "task_key": "fetch_population",
        "depends_on": [{"task_key": "sync_bls_data"}],
        "notebook_task": {
          "notebook_path": "/Users/rebeccalwatkins@gmail.com/rearc_quest/Databricks Version/Rearc Databricks - Section 2",
          "base_parameters": {
            "catalog": "rearcquest",
            "schema": "raw",
            "volume": "population",
            "sourceUrl": "https://honolulu-api.datausa.io/tesseract/data.jsonrecords?cube=acs_yg_total_population_1&drilldowns=Year%2CNation&locale=en&measures=Population"
          }
        },
        "new_cluster": {
          "spark_version": "15.4.x-scala2.12",
          "node_type_id": "Standard_DS3_v2",
          "num_workers": 1
        }
      },
      {
        "task_key": "run_analysis",
        "depends_on": [{"task_key": "fetch_population"}],
        "notebook_task": {
          "notebook_path": "/Users/rebeccalwatkins@gmail.com/rearc_quest/Databricks Version/Rearc Databricks - Section 3",
          "base_parameters": {
            "catalog": "rearcquest",
            "schema": "default"
          }
        },
        "new_cluster": {
          "spark_version": "15.4.x-scala2.12",
          "node_type_id": "Standard_DS3_v2",
          "num_workers": 2
        }
      }
    ],
    "schedule": {
      "quartz_cron_expression": "0 0 2 * * ?",
      "timezone_id": "UTC"
    },
    "email_notifications": {
      "on_failure": ["rebeccalwatkins@gmail.com"]
    }
  }'
```